# 读写文件

有时希望保存训练模型，以备在各种环境中使用（不如再部署中进行预测）。此外，当运行一个耗时较长的训练过程时， 最佳的做法是定期保存中间结果， 以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。 因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## 1. 加载和保存张量

In [2]:
'''对于单个张量，直接使用load和save函数分别读写
都需要提供一个名称，save要求将保存的变量作为输入
'''
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [3]:
'''可以将存储在文件中的数据读回内存'''
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [4]:
'''存储一个张量列表，将其读回内存'''
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [5]:
'''甚至可以写入或读取从字符串映射到张量的字典
读取或写入模型中的所有权重时十分方便
'''
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 2. 加载和保存模型参数

深度学习框架提供了内置函数保存和加载整个网络

保存的是模型的参数，而不是整个模型

如果有一个三层MLP，我们需要单独制定架构，因为模型本身可以包含人意代码，所以模型难以序列化

为了恢复模型，我们需要用代码生成架构，然后从磁盘加载参数

In [7]:
class  MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))
    
net = MLP()
X = torch.randn(size = (2, 20))
Y = net(X)

In [9]:
'''将模型参数存储在mlp.params文件中'''
torch.save(net.state_dict(), 'mlp.params') # state_dict()获取模型的状态字典，包括所有可学习的参数(weight, bias)

In [10]:
'''为了恢复模型，我们实例化了原始多层感知机模型的一个备份
我们不需要随机初始化模型参数，而是直接读取文件中存储的参数
'''
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [11]:
'''由于两个实例具有相同的模型参数
在输入相同X时，两个实例的计算结果应该相同，下面进行验证
'''
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])